# Machine Learning Poisonong Attacks in Malware Detection

## Prestart
### Import libraries

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers

### Files
Store the paths for our files in variables

In [4]:
file_arm_attacker_training = "files\\arm_datasets\\arm_attacker_training.csv"
file_arm_attacker_validation = "files\\arm_datasets\\arm_attacker_validation.csv"
file_arm_attacker_test = "files\\arm_datasets\\arm_attacker_test.csv"

file_arm_victim_training = "files\\arm_datasets\\arm_victim_training.csv"
file_arm_victim_validation = "files\\arm_datasets\\arm_victim_validation.csv"
file_arm_victim_test = "files\\arm_datasets\\arm_victim_test.csv"

## Creating the arm datasets
$ 8:1:1 $ - Training:Test:Validation

### Attacker dataset
For the first model - we call it *Clean* - we will train on clean a clean dataset

#### Reading in
In this state the csv contains the filename and the label

In [5]:
arm_attacker_training = pd.read_csv(file_arm_attacker_training, header=None, index_col=False)
arm_attacker_validation = pd.read_csv(file_arm_attacker_validation, header=None, index_col=False)
arm_attacker_test = pd.read_csv(file_arm_attacker_test, header=None, index_col=False)

#### Dataset
Creating the pure dataset, only features

In [6]:
dataset_arm_attacker_training = np.asarray(arm_attacker_training.drop(columns=arm_attacker_training.columns[-2:]))
dataset_arm_attacker_validation = np.asarray(arm_attacker_validation.drop(columns=arm_attacker_validation.columns[-2:]))
dataset_arm_attacker_test = np.asarray(arm_attacker_test.drop(columns=arm_attacker_test.columns[-2:]))

#### Labels for the attacker datasets
Creating the labels for the attacker datasets
*Malware* - $ 0 $, *Benign* - $ 1 $
-**Trainining**: 800 malware, 800 benign
-**Validation**: 100 malware, 100 benign
-**Test**: 100 malware, 100 benign

In [7]:
labels_arm_attacker_training = np.asarray(arm_attacker_training[arm_attacker_training.columns[-1]])
labels_arm_attacker_validation = np.asarray(arm_attacker_validation[arm_attacker_validation.columns[-1]])
labels_arm_attacker_test = np.asarray(arm_attacker_test[arm_attacker_test.columns[-1]])

#### Filenames
Getting the filenames separated just for the fun of it

In [8]:
names_arm_attacker_training = np.asarray(arm_attacker_training[arm_attacker_training.columns[-2]])
names_arm_attacker_validation = np.asarray(arm_attacker_validation[arm_attacker_validation.columns[-2]])
names_arm_attacker_test = np.asarray(arm_attacker_test[arm_attacker_test.columns[-2]])

### Victim dataset
For the second model - we call it *Victim* - we will train on clean a poisoned dataset

#### Reading in
In this state the csv contains the filename and the label

In [9]:
arm_victim_training = pd.read_csv(file_arm_victim_training, header=None, index_col=False)
arm_victim_validation = pd.read_csv(file_arm_victim_validation, header=None, index_col=False)
arm_victim_test = pd.read_csv(file_arm_victim_test, header=None, index_col=False)

#### Dataset
Creating the pure dataset, only features

In [10]:
dataset_arm_victim_training = np.asarray(arm_victim_training.drop(columns=arm_victim_training.columns[-2:]))
dataset_arm_victim_validation = np.asarray(arm_victim_validation.drop(columns=arm_victim_validation.columns[-2:]))
dataset_arm_victim_test = np.asarray(arm_victim_test.drop(columns=arm_victim_test.columns[-2:]))

#### Labels for the victim datasets
Creating the labels for the victim datasets
*Malware* - $ 0 $, *Benign* - $ 1 $
-**Trainining**: 800 malware, 800 benign
-**Validation**: 100 malware, 100 benign
-**Test**: 100 malware, 100 benign

In [11]:
labels_arm_victim_training = np.asarray(arm_victim_training[arm_victim_training.columns[-1]])
labels_arm_victim_validation = np.asarray(arm_victim_validation[arm_victim_validation.columns[-1]])
labels_arm_victim_test = np.asarray(arm_victim_test[arm_victim_test.columns[-1]])

#### Filenames
Getting the filenames separated just for the fun of it

In [12]:
names_arm_victim_training = arm_victim_training[arm_victim_training.columns[-2]]
names_arm_victim_validation = arm_victim_validation[arm_victim_validation.columns[-2]]
names_arm_victim_test = arm_victim_test[arm_victim_test.columns[-2]]

## Models for arm Datasets
In this section we will build the models with the exact same struture
Neural network with one *hidden layer* with a $sigmoid$ *activation function*

### Model for Attacker Data
Building, fitting and evaluating the Model with the *attacker* datasets

In [13]:
model_attacker = keras.Sequential(
    [
        layers.Dense(1, input_shape=(131,), activation="sigmoid")
    ]
)
model_attacker.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.BinaryAccuracy()])
model_attacker.fit(dataset_arm_attacker_training, labels_arm_attacker_training, epochs=10,
                validation_data=(dataset_arm_attacker_validation, labels_arm_attacker_validation))

model_attacker.evaluate(dataset_arm_attacker_test, labels_arm_attacker_test)

Epoch 1/10
50/50 [==============================] - 1s 5ms/step - loss: 3.7279 - binary_accuracy: 0.4925 - val_loss: 2.2140 - val_binary_accuracy: 0.4950
Epoch 2/10
50/50 [==============================] - 0s 2ms/step - loss: 1.7400 - binary_accuracy: 0.5213 - val_loss: 1.0480 - val_binary_accuracy: 0.5550
Epoch 3/10
50/50 [==============================] - 0s 2ms/step - loss: 0.7922 - binary_accuracy: 0.6181 - val_loss: 0.5538 - val_binary_accuracy: 0.7300
Epoch 4/10
50/50 [==============================] - 0s 2ms/step - loss: 0.4717 - binary_accuracy: 0.7744 - val_loss: 0.3983 - val_binary_accuracy: 0.8150
Epoch 5/10
50/50 [==============================] - 0s 2ms/step - loss: 0.3419 - binary_accuracy: 0.8562 - val_loss: 0.2981 - val_binary_accuracy: 0.8800
Epoch 6/10
50/50 [==============================] - 0s 2ms/step - loss: 0.2730 - binary_accuracy: 0.9013 - val_loss: 0.2505 - val_binary_accuracy: 0.9100
Epoch 7/10
50/50 [==============================] - 0s 2ms/step - loss: 0.22

[0.1752450317144394, 0.9399999976158142]

### Model for Victim Data
Building, fitting and evaluating the Model with the *victim* datasets

In [14]:
model_victim = keras.Sequential(
    [
        layers.Dense(1, input_shape=(131,), activation="sigmoid")
    ]
)
model_victim.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.BinaryAccuracy()])
model_victim.fit(dataset_arm_victim_training, labels_arm_victim_training, epochs=10,
                validation_data=(dataset_arm_victim_validation, labels_arm_victim_validation))

model_victim.evaluate(dataset_arm_victim_test, labels_arm_victim_test)

Epoch 1/10
50/50 [==============================] - 0s 4ms/step - loss: 0.7084 - binary_accuracy: 0.6744 - val_loss: 0.6180 - val_binary_accuracy: 0.7300
Epoch 2/10
50/50 [==============================] - 0s 2ms/step - loss: 0.4579 - binary_accuracy: 0.7713 - val_loss: 0.4454 - val_binary_accuracy: 0.8000
Epoch 3/10
50/50 [==============================] - 0s 2ms/step - loss: 0.3331 - binary_accuracy: 0.8512 - val_loss: 0.3280 - val_binary_accuracy: 0.8650
Epoch 4/10
50/50 [==============================] - 0s 2ms/step - loss: 0.2541 - binary_accuracy: 0.9013 - val_loss: 0.2653 - val_binary_accuracy: 0.9000
Epoch 5/10
50/50 [==============================] - 0s 2ms/step - loss: 0.2080 - binary_accuracy: 0.9344 - val_loss: 0.2186 - val_binary_accuracy: 0.9350
Epoch 6/10
50/50 [==============================] - 0s 2ms/step - loss: 0.1749 - binary_accuracy: 0.9469 - val_loss: 0.1921 - val_binary_accuracy: 0.9450
Epoch 7/10
50/50 [==============================] - 0s 2ms/step - loss: 0.14

[0.08626759797334671, 0.9750000238418579]

## POISON IT

### Files

In [15]:
file_poison_arm_BM100 = "files\\poison_data\\poisoned_benign_malware100.csv"
file_poison_arm_MB100 = "files\\poison_data\\poisoned_malware_benign100.csv"

### Poisoned Datasets

#### Attacker dataset

In [16]:
poisoned_arm_attacker_training = pd.read_csv(file_poison_arm_BM100, index_col=False, header=None)
poisoned_arm_attacker_training = arm_attacker_training.append(poisoned_arm_attacker_training, ignore_index=True).sample(frac=1)
dataset_poisoned_arm_attacker_training = np.asarray(poisoned_arm_attacker_training.drop(columns=poisoned_arm_attacker_training.columns[-2:]))
labels_poisoned_arm_attacker_training = np.asarray(poisoned_arm_attacker_training[poisoned_arm_attacker_training.columns[-1]])

C:\Users\ZsZs\AppData\Local\Temp\ipykernel_6128\2305770692.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  poisoned_arm_attacker_training = arm_attacker_training.append(poisoned_arm_attacker_training, ignore_index=True).sample(frac=1)


### Models

#### Attacker model

In [17]:
model_attacker = keras.Sequential(
    [
        layers.Dense(1, input_shape=(131,), activation="sigmoid")
    ]
)
model_attacker.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.BinaryAccuracy()])
model_attacker.fit(dataset_poisoned_arm_attacker_training, labels_poisoned_arm_attacker_training, epochs=10,
                validation_data=(dataset_arm_attacker_validation, labels_arm_attacker_validation))

model_attacker.evaluate(dataset_arm_attacker_test, labels_arm_attacker_test)

Epoch 1/10
51/51 [==============================] - 1s 4ms/step - loss: 1.8990 - binary_accuracy: 0.5913 - val_loss: 0.8727 - val_binary_accuracy: 0.6700
Epoch 2/10
51/51 [==============================] - 0s 2ms/step - loss: 0.6701 - binary_accuracy: 0.6950 - val_loss: 0.5491 - val_binary_accuracy: 0.7650
Epoch 3/10
51/51 [==============================] - 0s 2ms/step - loss: 0.4357 - binary_accuracy: 0.7807 - val_loss: 0.4348 - val_binary_accuracy: 0.7850
Epoch 4/10
51/51 [==============================] - 0s 2ms/step - loss: 0.3301 - binary_accuracy: 0.8528 - val_loss: 0.3122 - val_binary_accuracy: 0.8800
Epoch 5/10
51/51 [==============================] - 0s 2ms/step - loss: 0.2580 - binary_accuracy: 0.9025 - val_loss: 0.2476 - val_binary_accuracy: 0.9200
Epoch 6/10
51/51 [==============================] - 0s 2ms/step - loss: 0.2120 - binary_accuracy: 0.9242 - val_loss: 0.2118 - val_binary_accuracy: 0.9350
Epoch 7/10
51/51 [==============================] - 0s 2ms/step - loss: 0.18

[0.14448142051696777, 0.9549999833106995]